In [0]:
!pip install pyspark

    100% |████████████████████████████████| 213.4MB 88kB/s 
    100% |████████████████████████████████| 204kB 29.4MB/s 
  Stored in directory: /root/.cache/pip/wheels/cd/54/c2/abfcc942eddeaa7101228ebd6127a30dbdf903c72db4235b23
Successfully built pyspark


In [0]:
from pyspark import SparkContext
sc = SparkContext()
from pyspark.sql import SQLContext
sqlc = SQLContext(sc)

In [0]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/spark-2.4.0/spark-2.4.0-bin-hadoop2.7.tgz
!tar xf spark-2.4.0-bin-hadoop2.7.tgz
!pip install -q findspark

In [0]:
from pyspark import SparkContext
sc = SparkContext()


In [0]:
from pyspark.sql import SQLContext
sqlc = SQLContext(sc)


Saving iris.csv to iris.csv


In [0]:
df = sqlc.read.format('com.databricks.spark.csv').options(header = 'true', inferschema = 'true').load('iris.csv')



In [0]:
df.show(5)

+-----------+----------+-----------+----------+-------+
|Sepallength|Sepalwidth|Petallength|Petalwidth|Species|
+-----------+----------+-----------+----------+-------+
|        5.1|       3.5|        1.4|       0.2| setosa|
|        4.9|       3.0|        1.4|       0.2| setosa|
|        4.7|       3.2|        1.3|       0.2| setosa|
|        4.6|       3.1|        1.5|       0.2| setosa|
|        5.0|       3.6|        1.4|       0.2| setosa|
+-----------+----------+-----------+----------+-------+
only showing top 5 rows



Exercise 1

In [0]:
from pyspark.sql.functions import isnan, when, count, col
df.select([count(when(isnan(c) | col(c).isNull(),
                c)).alias(c) for c in df.columns]).show()


+-----------+----------+-----------+----------+-------+
|Sepallength|Sepalwidth|Petallength|Petalwidth|Species|
+-----------+----------+-----------+----------+-------+
|          0|         0|          0|         0|      0|
+-----------+----------+-----------+----------+-------+



iris dataset does not have empty values by default. The dataset is a little different here


In [0]:
df.filter(col('Sepallength').isNull()).count()

0

Exercise 2

In [0]:
from pyspark.sql.functions import isnan, when, count, col

In [0]:
df.select([count(when(isnan(i) | col(i).isNull(), i)).alias(i) for i in df.columns]).show()

+-----------+----------+-----------+----------+-------+
|Sepallength|Sepalwidth|Petallength|Petalwidth|Species|
+-----------+----------+-----------+----------+-------+
|          0|         0|          0|         0|      0|
+-----------+----------+-----------+----------+-------+



In [0]:
df.describe().show(1)

+-------+-----------+----------+-----------+----------+-------+
|summary|Sepallength|Sepalwidth|Petallength|Petalwidth|Species|
+-------+-----------+----------+-----------+----------+-------+
|  count|        150|       150|        150|       150|    150|
+-------+-----------+----------+-----------+----------+-------+
only showing top 1 row



In [0]:
df.where(col('Sepallength').isNull()).show() #we do not have null values

+-----------+----------+-----------+----------+-------+
|Sepallength|Sepalwidth|Petallength|Petalwidth|Species|
+-----------+----------+-----------+----------+-------+
+-----------+----------+-----------+----------+-------+



In [0]:
df.select('Sepallength').dropna().count()

150

In [0]:
df.dropna().count()

150

Exercise 4

In [0]:
y = df.select('Sepallength','Petallength').fillna(1)

In [0]:
y.select([count(when(isnan(i) | col(i).isNull(), i)).alias(i) for i in y.columns]).show()

+-----------+-----------+
|Sepallength|Petallength|
+-----------+-----------+
|          0|          0|
+-----------+-----------+



In [0]:
z = df.fillna(1)

In [0]:
z.select([count(when(isnan(k) | col(k).isNull(), k)).alias(k) for k in z.columns]).show()

+-----------+----------+-----------+----------+-------+
|Sepallength|Sepalwidth|Petallength|Petalwidth|Species|
+-----------+----------+-----------+----------+-------+
|          0|         0|          0|         0|      0|
+-----------+----------+-----------+----------+-------+



Exercise 5

In [0]:
df.corr('Sepallength', 'Sepalwidth')

-0.10936924995064931

In [0]:
from pyspark.mllib.stat import Statistics
import pandas as pd


In [0]:
z = df.fillna(1)

In [0]:
a = z.drop('Species')

In [0]:
features = a.rdd.map(lambda row: row[0:])
correlation_matrix = Statistics.corr(features, method="pearson")


In [0]:
correlation_df = pd.DataFrame(correlation_matrix)

In [0]:
correlation_df.index, correlation_df.columns = a.columns, a.columns

In [0]:
correlation_df

,Sepallength,Sepalwidth,Petallength,Petalwidth
Sepallength,1.000000,-0.109369,0.871754,0.817954
Sepalwidth,-0.109369,1.000000,-0.420516,-0.356544
Petallength,0.871754,-0.420516,1.000000,0.962757
Petalwidth,0.817954,-0.356544,0.962757,1.000000
